<a href="https://colab.research.google.com/github/MiguelAngel9604/spark_exercice_1/blob/main/Pyspark_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pyspark
!pip install pyspark
# Pyspark stubs
!pip install pyspark-stubs

In [3]:
#Creating Spark session
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf

spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise2") \
    .getOrCreate()

In [8]:
df = spark.read.format("csv").option("header","true").load("/content/Data exercise 2 - Hoja 1.csv")
df.show()

+------------+-----------+
|it's my name| birth/dob.|
+------------+-----------+
|       angel| 03-09-2020|
|      miguel|   22-02-19|
|        jose|02-Aug-2022|
|        raul|29March2022|
|       pedro|Mar/2020/22|
+------------+-----------+

